In [39]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
complete_doc_set = pd.DataFrame()


driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
base_page = 'http://www14.tceq.texas.gov/iwud/document/index.cfm?fuseaction=ListDocumentsByType&COMMAND=LIST'
driver.get(base_page)
#CountyCode

audit_vars_css = 'td.iwud:nth-child(1)'
audit_vals_css = '.iwud+ td'
get_doc_id = 'table:nth-child(6) tr:nth-child(1) .iwuddata'
access_more_docs = '.iwudgreyrow~ tr a img'
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
#
# select by value 
#select.select_by_value('1')
#wait = WebDriverWait(driver, 10)
select_doc_type = Select(driver.find_element_by_id('DocType'))
select_doc_type.select_by_visible_text('ANNUAL REPORTS')
driver.find_element_by_css_selector('.iwudstatus+ td input').click()
#empty_audit_info = pd.DataFrame()

#empty_audit_info = pd.read_csv('../../input/tceq_audits/audit_doc_scrapes.csv',index_col=0)
should_restart = True
while should_restart:
    should_restart = False
    doc_id_links = driver.find_elements_by_css_selector('.iwud+ .iwud a')
    district_name_elements = driver.find_elements_by_css_selector('form+ table .iwud:nth-child(2)')
    site_id_elements = driver.find_elements_by_css_selector('.iwud:nth-child(3)')
    #for site in site_id[1:]:
    #    site_ids.append(site.text)
    site_ids = []
    district_names = []
    doc_ids = []
    doc_urls = []
    not_blank = []
    #for s in site_ids:
    #    not_blank.append(s != ' ')
    #doc_id_links = [i for indx,i in enumerate(doc_id_links) if not_blank[indx]]
    for d in range(len(doc_id_links)):
        doc_ids.append(doc_id_links[d].text)
        doc_urls.append(doc_id_links[d].get_attribute('href'))
        district_names.append(district_name_elements[d].text)
        site_ids.append(site_id_elements[d].text)
    temp_doc_set = pd.DataFrame({'District Name': district_names,'SITE_ID':site_ids, 
     'DOC_ID': doc_ids,'DOC_URL':doc_urls})
    complete_doc_set = complete_doc_set.append(temp_doc_set)
    if len(driver.find_elements_by_css_selector("img[alt='Next page']"))!=0:
        driver.find_element_by_name('next').click()
        should_restart = True
        continue

In [40]:
complete_doc_set.to_csv('../../input/tceq_audits/private_entity_financial_report_reference_list.csv')